### Write well, Im going to be using this for a long time

#### Data we need:
user input:
- investment amount, trading pair -> amt0, amt1
- start time and end time
- time period that you assume fixed swap price, swap volumes or liquidity positions
- upper and lower price
- pool_fee_rate

data from api:
- cprice of each time period (tick, 1.0001 ** i)
- L_pool at each time period at specific pool_fee_rate (liquidity?, or simply total X tokens + Y tokens in USD)
- Swap volume at each time period at specific pool_fee_rate (volumeUSD?)
- Gas cost to mint at each time period

--------------------------------------------------------------------------------------------------------------

#### Fees
The liquidity amount is calculated from the following numbers that describe a position: 
- amount of token 0 (amt0), amount of token 1 (amt1), 
- price (as x token 1's per token 0) at the upper limit of the position (upper), 
- price at the lower limit of the position (lower) 
- and the current swap price (cprice). 

Then liquidity (L_you?) for a position is calculated as follows:

Case 1: cprice <= lower
- liquidity = amt0 * (sqrt(upper) * sqrt(lower)) / (sqrt(upper) - sqrt(lower))

Case 2: lower < cprice <= upper
- liquidity is the min of the following two calculations:
- amt0 * (sqrt(upper) * sqrt(cprice)) / (sqrt(upper) - sqrt(cprice))
- amt1 / (sqrt(cprice) - sqrt(lower))

Case 3: upper < cprice
- liquidity = amt1 / (sqrt(upper) - sqrt(lower))

Resources
- liquidity can use this code: https://github.com/JNP777/UNI_V3-Liquitidy-amounts-calcs/blob/main/UNI_v3_funcs.py

Fee is calculated by:
- Fee income = (L_you/L_pool) * swap volume under fixed time period * pool_fee_rate/100
- L_you also should be for that specific ticks only, not the whole amount you provided for. Its not linear, its calculated from the 3 cases above
- Does Case1 and Case3's fee be 0 regardless?


reference: https://uniswapv3.flipsidecrypto.com/
- check my numbers with the reference from the website

----------------------------------------------------------------------------------------

#### Impermanent Loss (is this v2 or v3)
- IL (in %) = (2 sqrt(p) / (p+1) ) - 1
- where p = r_t1/r_t2
- and r_t is a price in b at time 1
- Net $ loss = total asset value in dollars at stake time * IL (in%)

reference: https://chainbulletin.com/impermanent-loss-explained-with-examples-math/#:~:text=Impermanent%20loss%20is%20the%20difference,is%20equal%20to%20200%20DAI

--------------------------------------------------------------------------------------------------------------

#### Other cost

Gas_costs_mint = 500000 gwei * gas_price at that time (??? double check actual cost)

### PNL/APR
-PNL = Acumulated Fees_accrued (dolar value at generation) - IL - Gas_costs_mint

-APR = PNL/Initial_capital*(age of the position / year time)

--------------------------------------------------------------------------------------------------------------

## Dependencies

In [1]:
import requests
import json
import pandas as pd
import math
import numpy as np

## Main Functions

In [2]:
# function to use requests.post to make an API call to the subgraph url
def run_query(q):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
                            '',json={'query': q})
    if request.status_code == 200:
        return request
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))
        
        
# turns requests into dataframe        
# def results_to_df(query_result):
#     json_data_ = json.loads(query_result.text)
#     df_data_ = json_data_['data']['pools']
#     df_ = pd.DataFrame(df_data_)

#     return df_

In [3]:
def get_token_id(symbol):
    
    # default should be first:10, in case there are more than 1 coins with the same symbol
    query_ = """ 
    {{
      tokens(first:1, where:{{symbol: "{}"}}) {{
        id
        symbol
        name
      }}
    }}""".format(symbol)
    
    # run query
    query_result_ = run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    print(' ')
    print('get_token_id: {}'.format(symbol))
    print(json_data_)
    
    # make sure only return 1 object
    if len(json_data_['data']['tokens']) == 1:
        token_id_ = json_data_['data']['tokens'][0]['id']
        return token_id_
        
    else:
        print(json_data_['data'])
        raise Exception('Returned number of token_ids != 1')

        
def get_pool_id(token0_id, token1_id, feeTier):
    query_ = """
    {{
      pools(first: 10, 
        where:{{token0: "{}",
        token1: "{}",
        feeTier:"{}" }}) 
      {{
        id
        token0{{symbol}}
        token1{{symbol}}
        feeTier
      }}
    }}""".format(token0_id, token1_id, feeTier)
    
    
    # run query
    query_result_ = run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    print('\n get_pool_id for feeTier: {}'.format(feeTier))
    print(json_data_)
    
    # make sure there is only 1 pool that matches exactly
    if len(json_data_['data']['pools']) == 1:
        pool_id_ = json_data_['data']['pools'][0]['id']
        return pool_id_
    else:
        print(json_data_['data'])
        raise Exception('Returned number of token_ids != 1')

        
    return json_data_

In [4]:
def get_poolDayDatas(pool_id, num_datapoints=1000):
    # input: pool_id
    # num_datapoints (must be multiple of max_request_)
    
    max_request_ = 1000
    quotient_ = math.floor(num_datapoints/max_request_)
#     remainder_ = num_datapoints%max_request_
            
    query_base_ = '''
    {{
      poolDayDatas(first:{},
      skip: {},
        where:{{ pool: "{}" }},
      orderBy:date,
      orderDirection: desc) 
      {{
        date
        tick
        liquidity
        volumeUSD
        pool{{
            token0{{
                symbol
            }}
            token1{{
                symbol
            }}
        }}
      }}
    }}'''
    
    poolDayDatas_array_ = []
    
    # query loop
    for i in range(quotient_):
        q_first_ = max_request_
        q_next_ = i*max_request_
        query_ = query_base_.format(q_first_, q_next_, pool_id)
        query_result_ = run_query(query_)
        json_data_ = json.loads(query_result_.text)
#         print(json_data_)
        poolDayDatas_array_ += json_data_['data']['poolDayDatas']
    
    print(' ')
    print('\n Queried PoolDayDatas, total of {} datapoints'.format(str(len(poolDayDatas_array_))))
#     print('example:')
#     print(poolDayDatas_array_[0])
    
    # array to dataframe
    df_ = pd.json_normalize(poolDayDatas_array_)
    df_.drop_duplicates(subset=['date']) 
    
    return df_

In [5]:
def get_poolHourDatas(pool_id, num_datapoints=3000):
    # input: pool_id
    # num_datapoints (must be multiple of max_request_)
    
    max_request_ = 1000
    quotient_ = math.floor(num_datapoints/max_request_)
#     remainder_ = num_datapoints%max_request_
            
    query_base_ = '''
    {{
      poolHourDatas(first:{},
      skip: {},
        where:{{ pool: "{}" }},
      orderBy:periodStartUnix,
      orderDirection: desc) 
      {{
        periodStartUnix
        pool{{
            token0{{
                symbol
            }}
            token1{{
                symbol
            }}
        }}
        liquidity
        sqrtPrice
        token0Price
        token1Price
        tick
        feeGrowthGlobal0X128
        feeGrowthGlobal1X128
        tvlUSD
        volumeToken0
        volumeToken1
        volumeUSD
        feesUSD
        txCount
        open
        high
        low
        close
      }}
    }}'''
    
    poolDayDatas_array_ = []
    
    # query loop
    for i in range(quotient_):
        q_first_ = max_request_
        q_next_ = i*max_request_
        query_ = query_base_.format(q_first_, q_next_, pool_id)
        query_result_ = run_query(query_)
        json_data_ = json.loads(query_result_.text)
#         print(json_data_)
        try:
            poolDayDatas_array_ += json_data_['data']['poolHourDatas']
        except Exception:
#             print('.. Pass')
            pass
    
    print(' ')
    print('\n Queried poolHourDatas, total of {} datapoints'.format(str(len(poolDayDatas_array_))))
#     print('example:')
#     print(poolDayDatas_array_[0])
    
    # array to dataframe
    df_ = pd.json_normalize(poolDayDatas_array_)
#     df_.drop_duplicates(subset=['periodStartUnix']) # TODO: BUGGGG ??
    
    return df_

In [34]:
def get_swaps(pool_id, time_start='1627369200', time_end='1623772800', num_datapoints=6000):
    # input: pool_id
    # num_datapoints (must be multiple of max_request_)
    
    max_request_ = 1000
    quotient_ = math.floor(num_datapoints/max_request_)
#     remainder_ = num_datapoints%max_request_
           
    query_base_ = '''
    {{
      swaps(first:{}, skip: {},
            where:{{ pool: "{}",
            timestamp_lt: "{}",
            timestamp_gt: "{}"}},
          orderBy:timestamp,
          orderDirection: desc){{
        transaction {{
          blockNumber
          timestamp
          gasUsed
          gasPrice
        }}
        id
        timestamp
        tick
        amount0
        amount1
        amountUSD
        sqrtPriceX96
      }}
    }}'''
    
    swap_arrays_ = []
    
    # query loop
    for i in range(quotient_):
        q_first_ = max_request_
        q_next_ = i*max_request_
        query_ = query_base_.format(q_first_, q_next_, pool_id, time_start, time_end)
        query_result_ = run_query(query_)
        json_data_ = json.loads(query_result_.text)
        
        try:
            swap_arrays_ += json_data_['data']['swaps']
            
        except Exception:
            pass
        
    print('Queried Swaps, total of {} datapoints'.format(str(len(swap_arrays_))))
    print(' ')
    df_ = pd.json_normalize(swap_arrays_)
    
    # next time start, if at the edge, then we keep looping and skipping
    if len(swap_arrays_) != 0:
        # last element of timestamp, add 1 so next iterations still includes it
        next_time_start_ = str( int(df_['timestamp'][df_.index[-1]]) + 1 )
    else:
        next_time_start_ = time_start
            
            
    return df_, next_time_start_


# get_swap can only request 6000 datapoints at the time. this is to loop get_swap to get more data
def get_swaps_loop(pool_id, time_start='1627369200', time_end='1623772800'): # ,  num_datapoints= 150000
    
    print('time_start: {}, time_end: {}'.format(time_start, time_end))
    
    max_num_query = 6000
#     num_iterations = math.floor(num_datapoints/max_num_query) + 3 # add 3 just in case
    
    next_time_start_ = time_start
    count = 0 # counting number of times that data returns is less than maximum, meaning reaching the end
#     for i in range(num_iterations):
    first_time = True
    while(count<10):
        print('next_time_start_: {}'.format(next_time_start_))
        df_, next_time_start_ = get_swaps(pool_id, next_time_start_, time_end, num_datapoints=max_num_query)
        
        if first_time == True:
            df_all_ = df_
            first_time = False
        else:
            df_all_ = df_all_.append(df_)
            
        if df_.shape[0] < 6000:
            count += 1
    
    # drop duplicates, reset index
    df_all_ = df_all_.drop_duplicates(subset=['id'])
    df_all_ = df_all_.reset_index(drop=True)
    print('Total swaps data = {}'.format(df_all_.shape[0]))
    
    return df_all_

In [7]:
def merge_poolHourData_swaps_all(df_poolHourDatas, df_swaps_all):
    
    # Match timestamp with hour period, and assign to df_swaps_all['periodStartUnix']
    def compute_periodStartUnix(row_):
        return row_['timestamp'] - (row_['timestamp'] % 3600)
    def compute_periodEndUnix(row_):
        return row_['periodStartUnix'] + 3600

    df_swaps_all['timestamp'] = df_swaps_all['timestamp'].astype(int)    
    df_swaps_all['periodStartUnix'] = df_swaps_all.apply(lambda row: compute_periodStartUnix(row), axis=1)                       
    df_swaps_all['periodEndUnix'] = df_swaps_all.apply(lambda row: compute_periodEndUnix(row), axis=1)                       

    df_swaps_all['periodStartUnix'] = df_swaps_all['periodStartUnix'].astype(int)        
    df_swaps_all['periodEndUnix'] = df_swaps_all['periodEndUnix'].astype(int)
    
    # Create swaps_txCount to compare with txCount in poolHourDatas to check integrity
    df_swaps_all['swaps_txCount'] = 1

    # Groupby->Sum based on periodStartUnix, specify columns to sum at GROUPBY_COLS
    GROUPBY_COLS = ['periodStartUnix','amount0', 'amount1', 'amountUSD', 'swaps_txCount']
    df_swaps_to_merge = df_swaps_all[GROUPBY_COLS]
    df_swaps_to_merge = df_swaps_to_merge.astype({'periodStartUnix': 'int',
                                                 'amount0':'float','amount1':'float', 
                                                  'amountUSD':'float', 'swaps_txCount':'int'})
    df_swaps_to_merge = df_swaps_to_merge.groupby(by=['periodStartUnix']).sum()

    # Merge df_swaps_all (groupby) with df_poolHourDatas
    df_poolHourDatas['periodStartUnix'] = df_poolHourDatas['periodStartUnix'].astype(int)
    df_merged = df_poolHourDatas.merge(df_swaps_to_merge, how='left', on='periodStartUnix')
    df_merged['txCount'] = df_merged['txCount'].astype(int)
    
    return df_merged

In [8]:
def get_data(token0, token1, feeTier):
    
    # Indicate Tokens and FeeTier
    token0_id = get_token_id(TOKEN0)
    token1_id = get_token_id(TOKEN1)
    pool_id = get_pool_id(token0_id, token1_id, feeTier)

    # Get poolHourDatas
    df_poolHourDatas = get_poolHourDatas(pool_id, num_datapoints=10000)

    # Get Swap Datas within the poolHourDatas timeframe
    time_start = df_poolHourDatas['periodStartUnix'][0]
    time_end = df_poolHourDatas['periodStartUnix'][df_poolHourDatas.index[-1]]
    
    # Get Swaps data
    df_swaps_all = get_swaps_loop(pool_id, time_start, time_end) # ,  num_datapoints= 150000
    
    # Saving settings
    SETTINGS = '{}-{}-{}-timestamp-{}-{}.csv'.format(TOKEN0, TOKEN1, feeTier, time_start, time_end)
    df_swaps_all.to_csv('../data/df_swaps_all_'+SETTINGS)
    df_poolHourDatas.to_csv('../data/df_poolHourDatas_'+SETTINGS)
    
    # Merge data
    df_merged = merge_poolHourData_swaps_all(df_poolHourDatas, df_swaps_all)
    
    df_merged.to_csv('../data/df_merged_tmp_'+SETTINGS)
    
    return df_merged

### Test running

In [40]:
TOKEN0 = 'USDC'
TOKEN1 = 'WETH'
feeTier = '3000' 

df_merged = get_data(TOKEN0, TOKEN1, feeTier)

 
get_token_id: USDC
{'data': {'tokens': [{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'name': 'USD Coin', 'symbol': 'USDC'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', 'token0': {'symbol': 'USDC'}, 'token1': {'symbol': 'WETH'}}]}}
 

 Queried poolHourDatas, total of 2047 datapoints
time_start: 1627574400, time_end: 1620169200
next_time_start_: 1627574400
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1626880100
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1626076836
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1625213581
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1624659927
Queried Swaps, total of 6000 datapoints
 
next_time_start_: 1624301091
Queried Swaps, total of 6000 datapoints
 
next_time_sta

In [42]:
watch_list = ['periodStartUnix',  
              'txCount', 'swaps_txCount', # check data integrity
              'amount0', 'amount1', 'amountUSD', # swaps data
              'tick', 'liquidity', 'sqrtPrice', 'tvlUSD', # pool data at that hour
              'pool.token0.symbol', 'pool.token1.symbol', # token data
              'token0Price', 'token1Price'
             ]
df_merged[watch_list].sample(15)

,periodStartUnix,txCount,swaps_txCount,amount0,amount1,amountUSD,tick,liquidity,sqrtPrice,tvlUSD,pool.token0.symbol,pool.token1.symbol,token0Price,token1Price
741,1624906800,73,35.0,5.530299e+06,-2614.050611,1.050590e+07,199724,14587060849651540914,1720358097882360923287689141879718,270745062.3674130427724659610382269,USDC,WETH,2120.906169181695470952507356677086,0.0004714965775151796483344964248298871
456,1625932800,47,25.0,4.639630e+06,-2169.000242,6.706512e+06,199632,21083036761323270221,1712460283044960577892297886829770,291887912.9879411039389529703752241,USDC,WETH,2140.514391144559814762100337733511,0.0004671774243317689136141688938041066
971,1624078800,64,44.0,8.312868e+06,-3745.571407,8.765979e+06,199216,20756128009605257588,1677224978481696150949634185396149,350307249.309794325649954382771558,USDC,WETH,2231.395352743611261681056721617868,0.0004481500773811554284962657803747602
1121,1623538800,51,31.0,-1.741506e+06,736.007590,3.126347e+06,198590,24859820773732446023,1625559951166619818675596200657182,386616813.7272828361920525145653613,USDC,WETH,2375.489884396221400434463159747477,0.0004209658001781683618057418553749523
675,1625144400,53,33.0,-2.441209e+06,1155.379245,5.159369e+06,199734,14768215727359822229,1721284381851143342045338347258490,262991497.5564948316065257459002864,USDC,WETH,2118.624114217911785976415553494584,0.0004720044453799437252692616986986034
1577,1621897200,143,103.0,6.292276e+06,-2387.735825,1.796228e+07,197529,10692258100265286531,1541555222222497633203885652922269,199346270.0979027527802293324549475,USDC,WETH,2641.441482009341933405135066635895,0.0003785811674462312846896068469237063
1141,1623466800,110,87.0,5.622459e+06,-2431.356772,2.302591e+07,198934,24887993228837329685,1653761493597893114793744557760335,388249122.3657914851812251174593774,USDC,WETH,2295.162378021859420844118355798189,0.0004356990205032351187090823041415835
340,1626350400,73,54.0,-2.310734e+06,1232.811964,1.346020e+07,200829,10922381668698204164,1818084876612429175261280689196660,251940710.4986872315279468963007767,USDC,WETH,1899.025773548608688357968095356627,0.0005265857967432180062048939775057679
636,1625284800,33,15.0,1.826775e+06,-855.555322,1.823810e+06,199663,17183526124515552567,1715185490843976237202345893129222,265803506.873322664790885630234972,USDC,WETH,2133.717794126473881484463226054435,0.0004686655389727354217153377589905489
103,1627203600,26,9.0,-1.134217e+05,53.006333,5.380408e+05,199495,15999621149684838116,1700807012084683135499994994614078,273146684.5666044247416320780044286,USDC,WETH,2169.946827994254913551266503881825,0.0004608407851745976377214392670497954


In [44]:
# Eye check:
# print(df_poolHourDatas.shape)
# print(df_swaps_all.shape)
print(df_merged.shape)

print(df_merged['txCount'].sum())
print(df_merged['swaps_txCount'].sum())

(2047, 24)
149168
102317.0


##### Query tokens with symbol
{
  tokens(first:10, where:{symbol: "WETH"}) {
    id
    symbol
    name
  }
}

##### Query pools with token0 id,  token1 ids and feeTiers
{
  pools(first:10, 
    where:{token0:"0x6b175474e89094c44da98b954eedeac495271d0f",
    token1: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
    feeTier:"3000" }) 
  {
    id
    token0{symbol}
    token1{symbol}
    feeTier
  }
}

##### Query poolDayDatas with pool id, order by date - Needs to be iterative (max 1000 query)
{
  poolDayDatas(first:1000,
  next: 1000,
    where:{pool:"0xa80964c5bbd1a0e95777094420555fead1a26c1e"},
  orderBy:date,
  orderDirection: desc) 
  {
    date
    tick
    liquidity
    volumeUSD
  }
}





##### Query examples on filtering

{
  pools
  (first: 10, 
    where: {liquidity_gt: "1000000", 
      feeTier: "10000"}
    orderBy: liquidity, 
    orderDirection: desc)
  {
    token0{symbol}
    token1{symbol}
    liquidity
  }


(token0) DAI id = 0x6b175474e89094c44da98b954eedeac495271d0f
(token1) WETH id = 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
(feeTier) "3000"

(DAI-WETH 500) Pool id = 0x60594a405d53811d3bc4766596efd80fd545a270
(DAI-WETH 3000) Pool id = 0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8
(DAI-WETH 1000) Pool id = 0xa80964c5bbd1a0e95777094420555fead1a26c1e




In [ ]:
pool_id